In [45]:
import os

# scipy basics
import numpy as np
from osgeo import gdal      # Necessary to do this import to get rasterio to import
import rasterio
import rasterio.features
import rasterio.transform
import rasterio.windows

import time

# dask/parallelization libraries
import coiled
import dask
import dask.array as dar
from dask.distributed import Client, LocalCluster
from dask.distributed import print as dask_print  
import rioxarray
import xarray as xr

from numba import jit
import concurrent.futures

import boto3

import math

<font size="6">Making cloud and local clusters</font> 

In [2]:
coiled_cluster = coiled.Cluster(
    n_workers=10,
    use_best_zone=True, 
    compute_purchase_option="spot_with_fallback",
    idle_timeout="20 minutes",
    # name="DGibbs Europe height flux model", 
    account='jterry64'   # Necessary to use the AWS environment that Justin set up in Coiled
)

Output()

╭────────────────────────────────────────── Package Info ──────────────────────────────────────────╮
│                                ╷                                                                 │
│   Package                      │ Note                                                            │
│ ╶──────────────────────────────┼───────────────────────────────────────────────────────────────╴ │
│   coiled_local_carbon_notebook │ Source wheel built from                                         │
│                                │ C:\Users\david.gibbs\AppData\Local\anaconda3\envs\carbon_note   │
│                                │ book                                                            │
│                                ╵                                                                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

In [3]:
# Coiled cluster (cloud run)
coiled_client = coiled_cluster.get_client()
coiled_client

<Client: 'tls://10.1.11.30:8786' processes=5 threads=20, memory=74.17 GiB>

In [ ]:
# Local single-process cluster (local run). Will run .compute() on just one process, not a whole cluster.
local_client = Client()
local_client

<font size="6">Shutting down cloud and local clusters</font> 

In [ ]:
coiled_cluster.shutdown()

In [ ]:
local_client.shutdown()

<font size="6">Analysis</font> 

<font size="4">Paths and functions</font>

In [103]:
# General paths and constants

general_uri = 's3://gfw2-data/forest_change/GLAD_Europe_height_data/'

s3_out_dir = 'climate/European_height_carbon_model/outputs'

def timestr():
    return time.strftime("%Y%m%d_%H_%M_%S")

tile_size = 10      # Tile size in degrees is from the top left of the tile. 10 is a full tile. Anything smaller is a subset of that.

chunk_length = 8000

In [118]:
# Returns list of all chunk boundaries within a bounding box for chunks of a given size
def get_chunk_bounds(min_x, min_y, max_x, max_y, chunk_size):
    x, y = (min_x, min_y)
    chunks = []

    # Polygon Size
    while y < max_y:
        while x < max_x:
            bounds = [
                x,
                y,
                x + chunk_size,
                y + chunk_size,
            ]
            chunks.append(bounds)
            x += chunk_size
        x = min_x
        y += chunk_size

    return chunks

# Returns the encompassing tile_id string in the form YYN/S_XXXE/W based on a coordinate
def xy_to_tile_id(top_left_x, top_left_y):

    lat_ceil = math.ceil(top_left_y/10.0) * 10
    lng_floor = math.floor(top_left_x/10.0) * 10
    
    lng: str = f"{str(lng_floor).zfill(3)}E" if (lng_floor >= 0) else f"{str(-lng_floor).zfill(3)}W"
    lat: str = f"{str(lat_ceil).zfill(2)}N" if (lat_ceil >= 0) else f"{str(-lat_ceil).zfill(2)}S"

    return f"{lat}_{lng}"

In [93]:
# Lazily opens tile within provided bounds (one chunk)
def get_tile_dataset_rio(uri, bounds):
    try:
        with rasterio.open(uri) as ds:
            return ds.read(1, window=rasterio.windows.from_bounds(*bounds, ds.transform))
    except rasterio.errors.RasterioIOError as e:
        return np.zeros((chunk_length, chunk_length))

<font size="4">Model steps</font>

In [114]:
# Opeates on pixels in each chunk for a given year: forest state, emission factor, removal factor, emissions, removals, carbon stocks
@jit(nopython=True)
def classify(forest_height_previous_block, forest_height_current_block, forest_loss_detection_block, 
             driver_block, planted_forest_type_block, peat_block, tclf_block, agb_2000_block, soil_c_2000_block,
            land_cover_previous_block, land_cover_next_block, burned_area_two_before_block, burned_area_one_before_block):

    # Output chunks
    forest_states = np.zeros(forest_height_previous_block.shape)
    emission_factor = np.zeros(forest_height_previous_block.shape)
    removal_factor = np.zeros(forest_height_previous_block.shape)
    emissions_agb = np.zeros(forest_height_previous_block.shape)
    emissions_soil = np.zeros(forest_height_previous_block.shape)

    # Iterates through all pixels in the chunk
    for row in range(forest_height_previous_block.shape[0]):
        for col in range(forest_height_previous_block.shape[1]):

            # Pixel for each input
            forest_height_previous = forest_height_previous_block[row, col]
            forest_height_current = forest_height_current_block[row, col]
            forest_loss_detection = forest_loss_detection_block[row, col]
            driver = driver_block[row, col]
            planted_forest_type = planted_forest_type_block[row, col]
            peat = peat_block[row, col]
            tclf = tclf_block[row, col]
            agb_2000 = agb_2000_block[row, col]
            soil_c_2000 = soil_c_2000_block[row, col]
            LC_previous = land_cover_previous_block[row, col]
            LC_next = land_cover_next_block[row, col]
            burned_area_two_before = burned_area_two_before_block[row, col]
            burned_area_one_before = burned_area_one_before_block[row, col]

            # Various definitions
            grassland_forest_previous = (((LC_previous >= 2) & (LC_previous <= 48)) | ((LC_previous >= 102) & (LC_previous <= 148)))
            grassland_forest_next = (((LC_next >= 2) & (LC_next <= 48)) | ((LC_next >= 102) & (LC_next <= 148)))  
            cropland_previous = (LC_previous == 244)
            cropland_next = (LC_next == 244)
            forestry = (driver == 3)
            non_sdpt_forestry = (forestry & (grassland_forest_previous | grassland_forest_next) & (cropland_previous == 0) & (cropland_next == 0))

            burned_area_recent = ((burned_area_two_before != 0) or (burned_area_one_before != 0))

            # create dictionary of forest states to emission factors: look up the emission factor for the forest state

            # The decision tree that produces all the outputs for a pixel for a given year
            if forest_height_previous >= 5 and forest_height_current >= 5:                                     # maintained
                rf = 0.1
                forest_states[row, col] = 1
                removal_factor[row, col] = rf
                emissions_agb[row, col] = agb_2000 * rf
                emissions_soil[row, col] = soil_c_2000 * rf
            elif forest_height_previous < 5 and forest_height_current >= 5:                                    # gain
                rf = 0.3
                forest_states[row, col] = 2
                removal_factor[row, col] = rf
                emissions_agb[row, col] = agb_2000 * rf
                emissions_soil[row, col] = soil_c_2000 * rf
            elif ((forest_height_previous >= 5 and forest_height_current < 5) or forest_loss_detection == 1):  # loss
                if planted_forest_type == 0:                                                                   # loss:no SDPT                
                    if non_sdpt_forestry == 0:                                                                 # loss:no SDPT:no non-SDPT forestry                    
                        ef = 0.9
                        forest_states[row, col] = 311
                        emission_factor[row, col] = ef
                        emissions_agb[row, col] = agb_2000 * ef
                        emissions_soil[row, col] = soil_c_2000 * ef 
                    else:                                                                                      # loss:no SDPT:non-SDPT forestry         
                        ef = 0.7
                        forest_states[row, col] = 312
                        emission_factor[row, col] = ef
                        emissions_agb[row, col] = agb_2000 * ef
                        emissions_soil[row, col] = soil_c_2000 * ef 
                else:                                                                                          # loss:SDPT
                    if burned_area_recent == 0:                                                                # loss:SDPT:not burned recent
                        if peat == 0:                                                                          # loss:SDPT:not burned recent:not peat
                            ef = 0.6
                            forest_states[row, col] = 3211
                            emission_factor[row, col] = ef
                            emissions_agb[row, col] = agb_2000 * ef
                            emissions_soil[row, col] = soil_c_2000 * ef
                        else:                                                                                  # loss:SDPT:not burned recent:peat
                            ef = 0.75
                            forest_states[row, col] = 3212
                            emission_factor[row, col] = ef
                            emissions_agb[row, col] = agb_2000 * ef
                            emissions_soil[row, col] = soil_c_2000 * ef                                
                    else:                                                                                      # loss:SDPT:burned recent
                        if peat == 0:                                                                          # loss:SDPT:burned recent:not peat
                            ef = 0.65
                            forest_states[row, col] = 3221
                            emission_factor[row, col] = ef
                            emissions_agb[row, col] = agb_2000 * ef
                            emissions_soil[row, col] = soil_c_2000 * ef
                        else:                                                                                  # loss:SDPT:burned recent:peat
                            ef = 0.9
                            forest_states[row, col] = 3222
                            emission_factor[row, col] = ef
                            emissions_agb[row, col] = agb_2000 * ef
                            emissions_soil[row, col] = soil_c_2000 * ef
            else:                                                                                              # no forest
                forest_states[row, col] = 0
                emission_factor[row, col] = 0
                removal_factor[row, col] = 0
                emissions_agb[row, col] = 0
                emissions_soil[row, col] = 0

    return forest_states, emission_factor, removal_factor, emissions_agb, emissions_soil

In [116]:
def process_chunk(bounds):
    
    futures = {}
    layers = {}

    bounds_str = "_".join([str(round(x)) for x in bounds])

    # start_year = 2002
    start_year = 2021

    try:
        # submit requests to S3 for layers
        with concurrent.futures.ThreadPoolExecutor() as executor:
            tile_id = xy_to_tile_id(bounds[0], bounds[3])
            drivers_uri = f"s3://gfw2-data/climate/carbon_model/other_emissions_inputs/tree_cover_loss_drivers/processed/drivers_2022/20230407/{tile_id}_tree_cover_loss_driver_processed.tif"
            planted_forest_type_uri = f"s3://gfw2-data/climate/carbon_model/other_emissions_inputs/planted_forest_type/SDPT_v1/standard/20200730/{tile_id}_plantation_type_oilpalm_woodfiber_other_unmasked.tif"
            peat_uri = f"s3://gfw2-data/climate/carbon_model/other_emissions_inputs/peatlands/processed/20230315/{tile_id}_peat_mask_processed.tif"
            tclf_uri = f"s3://gfw2-data/climate/carbon_model/other_emissions_inputs/tree_cover_loss_fires/20230315/processed/{tile_id}_tree_cover_loss_fire_processed.tif"
            agb_2000_uri = f"s3://gfw2-data/climate/WHRC_biomass/WHRC_V4/Processed/{tile_id}_t_aboveground_biomass_ha_2000.tif"
            soil_c_2000_uri = f"s3://gfw2-data/climate/carbon_model/carbon_pools/soil_carbon/intermediate_full_extent/standard/20230222/{tile_id}_t_soil_C_ha_full_extent_2000.tif"

            # save as dictionary from future -> name so we can know what layer it is on completion
            futures[executor.submit(get_tile_dataset_rio, drivers_uri, bounds)] = "drivers"
            futures[executor.submit(get_tile_dataset_rio, planted_forest_type_uri, bounds)] = "planted_forest_type"
            futures[executor.submit(get_tile_dataset_rio, peat_uri, bounds)] = "peat"
            futures[executor.submit(get_tile_dataset_rio, tclf_uri, bounds)] = "tclf"
            futures[executor.submit(get_tile_dataset_rio, agb_2000_uri, bounds)] = "agb_2000"
            futures[executor.submit(get_tile_dataset_rio, soil_c_2000_uri, bounds)] = "soil_c_2000"

            # faster to just get every year up front unless we're running into memory issues
            for year in range(start_year-1, 2022):
                forest_height_uri = f'{general_uri}202307_revision/FH_{year}.tif'
                forest_loss_detection_uri = f'{general_uri}202307_revision/DFL_{year}.tif'

                preceding_land_cover_uri = f's3://gfw2-data/landcover/composite/2015/{tile_id}_composite_landcover_2015.tif'
                next_land_cover_uri = f's3://gfw2-data/landcover/composite/2020/{tile_id}_composite_landcover_2020.tif'

                futures[executor.submit(get_tile_dataset_rio, forest_height_uri, bounds)] = f"forest_height_{year}"
                futures[executor.submit(get_tile_dataset_rio, forest_loss_detection_uri, bounds)] = f"forest_loss_detection_{year}"
                
                futures[executor.submit(get_tile_dataset_rio, preceding_land_cover_uri, bounds)] = f"preceding_land_cover_for_{year}"
                futures[executor.submit(get_tile_dataset_rio, next_land_cover_uri, bounds)] = f"next_land_cover_for_{year}"

            for year in range(start_year-2, 2022):
                burned_area_uri = f's3://gfw2-data/climate/carbon_model/other_emissions_inputs/burn_year/burn_year_10x10_clip/ba_{year}_50N_010E.tif'

                futures[executor.submit(get_tile_dataset_rio, burned_area_uri, bounds)] = f"burned_area_{year}"

        # wait for requests to come back with data from S3
        for future in concurrent.futures.as_completed(futures):
            layer = futures[future]
            layers[layer] = future.result()

        # agb = agb_2000 # Make sure to reassign variable so it's not accumulating variables
        
        # run classifier one year at a time
        for year in range(start_year, 2022):
            dask_print(f"Classifying {bounds_str} in {tile_id} for {year}: {timestr()}")
            forest_states, emission_factor, removal_factor, emissions_agb, emissions_soil = classify(
                layers[f"forest_height_{year - 1}"], 
                layers[f"forest_height_{year}"], 
                layers[f"forest_loss_detection_{year}"], 
                layers["drivers"],
                layers["planted_forest_type"],
                layers["peat"],
                layers["tclf"],
                layers["agb_2000"],
                layers["soil_c_2000"],
                layers[f"preceding_land_cover_for_{year}"],
                layers[f"next_land_cover_for_{year}"],
                layers[f"burned_area_{year-2}"],
                layers[f"burned_area_{year-1}"],
            )
            
            transform = rasterio.transform.from_bounds(*bounds, width=chunk_length, height=chunk_length)
            
            states_file_name = f"forest_states__{tile_id}__{year}__{bounds_str}__{timestr()}"
            with rasterio.open(f"/tmp/{states_file_name}.tif", 'w', driver='GTiff', width=chunk_length, height=chunk_length, count=1, dtype='uint16', crs='EPSG:4326', transform=transform, compress='lzw', blockxsize=400, blockysize=400) as dst1:
                dst1.write(forest_states.astype(rasterio.uint16), 1)

            ef_file_name = f"emission_factor__{tile_id}__{year}__{bounds_str}__{timestr()}"
            with rasterio.open(f"/tmp/{ef_file_name}.tif", 'w', driver='GTiff', width=chunk_length, height=chunk_length, count=1, dtype='float32', crs='EPSG:4326', transform=transform, compress='lzw', blockxsize=400, blockysize=400) as dst2:
                dst2.write(emission_factor.astype(rasterio.float32), 1)

            rf_file_name = f"removal_factor__{tile_id}__{year}__{bounds_str}__{timestr()}"
            with rasterio.open(f"/tmp/{rf_file_name}.tif", 'w', driver='GTiff', width=chunk_length, height=chunk_length, count=1, dtype='float32', crs='EPSG:4326', transform=transform, compress='lzw', blockxsize=400, blockysize=400) as dst2:
                dst2.write(removal_factor.astype(rasterio.float32), 1)

            agb_emis_file_name = f"agb_emis_{tile_id}__{year}__{bounds_str}__{timestr()}"
            with rasterio.open(f"/tmp/{agb_emis_file_name}.tif", 'w', driver='GTiff', width=chunk_length, height=chunk_length, count=1, dtype='float32', crs='EPSG:4326', transform=transform, compress='lzw', blockxsize=400, blockysize=400) as dst3:
                dst3.write(emissions_agb.astype(rasterio.float32), 1)

            # soil_emis_file_name = f"soil_emis_{tile_id}__{year}__{bounds_str}__{timestr()}"
            # with rasterio.open(f"/tmp/{soil_emis_file_name}.tif", 'w', driver='GTiff', width=chunk_length, height=chunk_length, count=1, dtype='float32', crs='EPSG:4326', transform=transform, compress='lzw', blockxsize=400, blockysize=400) as dst4:
            #     dst4.write(emissions_soil.astype(rasterio.float32), 1)

            dask_print(f"Uploading {bounds_str} in {tile_id} for {year}: {timestr()}")
            s3_client = boto3.client("s3")
            s3_client.upload_file(f"/tmp/{states_file_name}.tif", "gfw2-data", Key=f"{s3_out_dir}/{states_file_name}.tif")

            s3_client.upload_file(f"/tmp/{ef_file_name}.tif", "gfw2-data", Key=f"{s3_out_dir}/{ef_file_name}.tif")

            s3_client.upload_file(f"/tmp/{rf_file_name}.tif", "gfw2-data", Key=f"{s3_out_dir}/{rf_file_name}.tif")

            s3_client.upload_file(f"/tmp/{agb_emis_file_name}.tif", "gfw2-data", Key=f"{s3_out_dir}/{agb_emis_file_name}.tif")

            # s3_client.upload_file(f"/tmp/{soil_emis_file_name}.tif", "gfw2-data", Key=f"{s3_out_dir}/{soil_emis_file_name}.tif")

            # clear memory of arrays we're done with
            del forest_states
            del emission_factor
            del removal_factor
            del emissions_agb
            del emissions_soil
            del layers[f"forest_height_{year - 1}"]
            del layers[f"forest_loss_detection_{year}"]
            layers[f"burned_area_{year-2}"]
    except Exception as e:
        return f"error: {e}"

    return f"success for {bounds_str}: {timestr()}"

In [117]:
%%time


"""
Some code that applies the decision tree to decision_tree_ds to make an xarray of forest_states for the previous and current years
"""

# chunks = get_chunk_bounds(-12, 34, 32, 72, 2)  # all of Europe. Final argument is chunk size in degrees.
chunks = get_chunk_bounds(6, 40, 16, 50, 2)  # smaller area that includes 50N_010E. Final argument is chunk size in degrees.
delayed = [dask.delayed(process_chunk)(chunk) for chunk in chunks]

results = dask.compute(*delayed)
results

Classifying 6_40_8_42 in 50N_000E for 2021: 20231103_20_17_07
Classifying 8_40_10_42 in 50N_000E for 2021: 20231103_20_17_08
Classifying 10_40_12_42 in 50N_010E for 2021: 20231103_20_17_08
Classifying 14_40_16_42 in 50N_010E for 2021: 20231103_20_17_09
Classifying 6_48_8_50 in 50N_000E for 2021: 20231103_20_17_11
Classifying 8_42_10_44 in 50N_000E for 2021: 20231103_20_17_12
Classifying 6_42_8_44 in 50N_000E for 2021: 20231103_20_17_12
Classifying 14_42_16_44 in 50N_010E for 2021: 20231103_20_17_12
Classifying 8_48_10_50 in 50N_000E for 2021: 20231103_20_17_11
Classifying 14_44_16_46 in 50N_010E for 2021: 20231103_20_17_13
Classifying 12_44_14_46 in 50N_010E for 2021: 20231103_20_17_13
Uploading 6_40_8_42 in 50N_000E for 2021: 20231103_20_17_17
Uploading 8_40_10_42 in 50N_000E for 2021: 20231103_20_17_17
Classifying 12_42_14_44 in 50N_010E for 2021: 20231103_20_17_13
Classifying 8_44_10_46 in 50N_000E for 2021: 20231103_20_17_13
Classifying 6_44_8_46 in 50N_000E for 2021: 20231103_20_1

('success for 6_40_8_42: 20231103_20_17_20',
 'success for 8_40_10_42: 20231103_20_17_27',
 'success for 10_40_12_42: 20231103_20_17_22',
 'success for 12_40_14_42: 20231103_20_17_53',
 'success for 14_40_16_42: 20231103_20_17_29',
 'success for 6_42_8_44: 20231103_20_17_28',
 'success for 8_42_10_44: 20231103_20_17_29',
 'success for 10_42_12_44: 20231103_20_18_00',
 'success for 12_42_14_44: 20231103_20_17_35',
 'success for 14_42_16_44: 20231103_20_17_31',
 'success for 6_44_8_46: 20231103_20_17_34',
 'success for 8_44_10_46: 20231103_20_17_34',
 'success for 10_44_12_46: 20231103_20_21_01',
 'success for 12_44_14_46: 20231103_20_17_25',
 'success for 14_44_16_46: 20231103_20_17_26',
 'success for 6_46_8_48: 20231103_20_17_29',
 'success for 8_46_10_48: 20231103_20_17_32',
 'success for 10_46_12_48: 20231103_20_18_21',
 'success for 12_46_14_48: 20231103_20_17_32',
 'success for 14_46_16_48: 20231103_20_18_18',
 'success for 6_48_8_50: 20231103_20_17_30',
 'success for 8_48_10_50: 2

In [ ]:
coiled_client.restart() 